# Collecting Data to CSV 

This notebook will be used to collect the data and transform it to a csv file if needed. 

In [1]:
%%capture
!pip install geopandas
!pip install pyreadstat

In [2]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import matplotlib.colors as colors
import warnings
import plotly.graph_objects as go
import geopandas as gpd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%autosave 120

Autosaving every 120 seconds


## Climate Data

In [3]:
def read_cc(pathway):
    df = pd.read_csv(pathway)
    state_name = df.iloc[:, 0].name
    df['State'] = state_name

    df = df.rename(columns={state_name: 'Date'})
    # Drop the first three rows
    df = df.drop([0, 1, 2])

    # Reset the index
    df = df.reset_index(drop=True)

    # Convert the 'Date' column 
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
    df['Date'] = df['Date'].dt.strftime('%Y-%m')

    return df

In [4]:
%%capture

df_ca_pdsi=read_cc('/project/ClimateData/CA-PDSI-1993-2023-3.csv')
df_ca_pre=read_cc('/project/ClimateData/CA-pre-1993-2023-2.csv')
df_ca_tem=read_cc('/project/ClimateData/CA-tem-1993-2023.csv')

# Merge the three dataframes based on 'Date' and 'State' columns
df_ca = pd.merge(df_ca_pdsi, df_ca_pre, on=['Date', 'State'])
df_ca = pd.merge(df_ca, df_ca_tem, on=['Date', 'State'])

In [5]:
# Save the dataframe to a CSV file
df_ca.to_csv("climate_change.csv", index=False)

In [6]:
df_ca

,Date,Palmer Drought Severity Index (PDSI),State,Precipitation,Average Temperature
0,1993-02,3.44,California,5.98,44.0
1,1993-03,2.90,California,3.05,52.9
2,1993-04,2.46,California,1.45,55.2
3,1993-05,2.68,California,1.44,62.3
4,1993-06,3.06,California,0.87,68.0
...,...,...,...,...,...
357,2022-11,-4.30,California,2.00,46.6
358,2022-12,-2.97,California,6.41,43.1
359,2023-01,-0.86,California,8.24,42.7
360,2023-02,-0.70,California,3.39,42.8


# Census data - county level 

In [6]:
def fetch_single_variable(year, variable):
    api_key = '49de4b7525f2760486308e3cbee0ed69c4f8cc65'
    base_url = f'https://api.census.gov/data/{year}/acs/acs5'

    # Create the request URL
    url = f'{base_url}?get=NAME,{variable}&for=county:*&in=state:06&key={api_key}'

    # Fetch the data
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        df['year'] = year
        return df
    else:
        print(f"Error fetching data for {year}, variable {variable}: {response.status_code}")
        return None


In [7]:
def fetch_names(year):
    url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME&for=county:*&in=state:06'
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error fetching names for year {year}: {response.status_code}")
        return None

    data = response.json()
    columns = data[0]
    values = data[1:]
    df = pd.DataFrame(values, columns=columns)
    df['year'] = year
    return df

In [8]:
def fetch_data(year, variables):
    name_df = fetch_names(year)
    
    if name_df is None:
        return None

    for variable in variables:
        temp_df = fetch_single_variable(year, variable)
        if temp_df is not None:
            print("Before merge:")
            print("name_df:")
            print(name_df)
            print("temp_df:")
            print(temp_df)
            name_df = name_df.merge(temp_df, on=['NAME', 'state', 'county', 'year'], how='left')
            print("After merge:")
            print(name_df)
        else:
            name_df[variable] = np.nan

    return name_df

In [9]:
%%capture
years = range(2011, 2023)

variables = [
    'B01003_001E',   # Total population
    'B11016_001E',   # Total households
    'B01001_026E',   # female population 
    'B01001_002E',   # male population 
    'B01002_001E',   # Median age 
    'B19013_001E',   # Median household income in the past 12 months
    'B15003_022E',   # Bachelor's degree 
    'B15003_023E',   # Master's degree 
    'B15003_024E',   # Professional degree
    'B15003_025E',   # Doctorate degree
    'B23025_004E',   # Number of employed people 
    'B23025_005E',   # Number of unemployed people 
    'C24030_004E', 'C24030_031E', # Number of employed people in agriculture/fishing/hunting
    'B23025_003E',   # Civilian labor force
    'B25010_001E',   # Average household size 
    'B25077_001E',   # Median value of owner-occupied housing units
    'B25064_001E',   # Median gross rent of renter-occupied units
    'B28002_013E',   # Households with no Internet access
    'B28002_002E',   # Households with an Internet subscription
    'B01001_003E', 'B01001_004E','B01001_005E','B01001_006E',   # Population under 18 years -male
    'B01001_007E', 'B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E',  # Population 18 to 34 years - male
    'B01001_013E', 'B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E',   # Population 35 to 64 years - male
    'B01001_020E', 'B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E',  # Population 65 years and over - male
    'B01001_027E', 'B01001_028E','B01001_029E','B01001_030E',   # Population under 18 years -female
    'B01001_031E', 'B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E',  # Population 18 to 34 years - female
    'B01001_037E', 'B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E',   # Population 35 to 64 years - female
    'B01001_044E', 'B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E',  # Population 65 years and over-female
    'B25040_008E', # Households with solar energy heating fuel 
]

# Fetch data for each year and store the dataframe 
data_frames = []
for year in years:
    data = fetch_data(year, variables)
    if data is not None:
        # Remove this line so the 'NAME', 'state', and 'county' columns are included in the final DataFrame
        data['year'] = year
        data_frames.append(data.reset_index(drop=True))

# Concatenate 
census_api = pd.concat(data_frames, ignore_index=True)

In [10]:
census_df_variables = census_api.copy()

In [11]:
census_df_variables

,NAME,state,county,year,B01003_001E,B11016_001E,B01001_026E,B01001_002E,B01002_001E,B19013_001E,...,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B25040_008E
0,"Merced County, California",06,047,2011,253606,74079,125935,127671,29.5,43945,...,5740,2290,3049,1573,2209,3081,2654,1933,1666,42
1,"Mariposa County, California",06,043,2011,18290,7607,9031,9259,48.4,49174,...,833,281,402,246,299,509,428,137,247,28
2,"Yuba County, California",06,115,2011,71817,23885,35565,36252,31.7,46617,...,1785,860,1032,440,630,880,565,837,454,10
3,"Contra Costa County, California",06,013,2011,1037817,370925,531217,506600,38.3,79135,...,36090,12543,16844,8685,11875,16108,12507,10975,12533,203
4,"Lassen County, California",06,035,2011,35001,10097,12016,22985,37.0,52484,...,990,366,408,227,346,289,305,262,286,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,"Tulare County, California",06,107,2021,470999,139631,234574,236425,31.3,57394,...,12544,4836,6270,4203,4862,7710,4334,3535,4222,1018
634,"Tuolumne County, California",06,109,2021,55243,23103,26488,28755,48.6,66846,...,2004,846,1648,930,1251,2155,1237,726,1144,303
635,"Ventura County, California",06,111,2021,845255,274471,424655,420600,38.7,94150,...,29320,11215,16322,10506,12653,17920,11658,8293,11334,1499
636,"Yolo County, California",06,113,2021,216703,75471,111335,105368,31.5,78386,...,5863,2370,3340,2123,2529,4101,2510,1635,2150,1865


In [12]:
census_df_variables.county.nunique() # There are 58 counties in California so this is correct

58

In [13]:
census_df_variables.year.unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [14]:
census_df_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 71 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   NAME         638 non-null    object
 1   state        638 non-null    object
 2   county       638 non-null    object
 3   year         638 non-null    int64 
 4   B01003_001E  638 non-null    object
 5   B11016_001E  638 non-null    object
 6   B01001_026E  638 non-null    object
 7   B01001_002E  638 non-null    object
 8   B01002_001E  638 non-null    object
 9   B19013_001E  638 non-null    object
 10  B15003_022E  580 non-null    object
 11  B15003_023E  580 non-null    object
 12  B15003_024E  580 non-null    object
 13  B15003_025E  580 non-null    object
 14  B23025_004E  638 non-null    object
 15  B23025_005E  638 non-null    object
 16  C24030_004E  638 non-null    object
 17  C24030_031E  638 non-null    object
 18  B23025_003E  638 non-null    object
 19  B25010_001E  638 non-null    

In [15]:
census_df_variables.iloc[:, 4:] = census_df_variables.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')

/tmp/ipykernel_133/3801879762.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  census_df_variables.iloc[:, 4:] = census_df_variables.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')


In [16]:
# Calculate the population under 18
census_df_variables['Num_under_18'] = census_df_variables[['B01001_003E', 'B01001_004E','B01001_005E','B01001_006E', 'B01001_027E', 'B01001_028E','B01001_029E','B01001_030E']].sum(axis=1)
# From 18 to 34
census_df_variables['Num_18_to_34'] = census_df_variables[['B01001_007E', 'B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E', 'B01001_031E', 'B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E']].sum(axis=1)
# From 35 to 64
census_df_variables['Num_35_to_64'] = census_df_variables[['B01001_013E', 'B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E', 'B01001_037E', 'B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E']].sum(axis=1)
# 65 and over
census_df_variables['Num_65_more'] = census_df_variables[['B01001_020E', 'B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E', 'B01001_044E', 'B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']].sum(axis=1)


In [17]:
census_df_variables['Num_industry_agriculture_fishing_hunting'] = census_df_variables[['C24030_004E','C24030_031E']].sum(axis=1)

In [18]:
census_df_variables.drop(columns=['B01001_003E', 'B01001_004E','B01001_005E','B01001_006E',   # Population under 18 years -male
    'B01001_007E', 'B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E',  # Population 18 to 34 years - male
    'B01001_013E', 'B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E',   # Population 35 to 64 years - male
    'B01001_020E', 'B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E',  # Population 65 years and over - male
    'B01001_027E', 'B01001_028E','B01001_029E','B01001_030E',   # Population under 18 years -female
    'B01001_031E', 'B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E',  # Population 18 to 34 years - female
    'B01001_037E', 'B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E',   # Population 35 to 64 years - female
    'B01001_044E', 'B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E',  # Population 65 years and over-female
                                  'C24030_004E', 'C24030_031E', # Number of employed people in agriculture/fishing/hunting'
                                 ], inplace = True)

In [19]:
cols = census_df_variables.columns.tolist()
print(cols)

['NAME', 'state', 'county', 'year', 'B01003_001E', 'B11016_001E', 'B01001_026E', 'B01001_002E', 'B01002_001E', 'B19013_001E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E', 'B23025_004E', 'B23025_005E', 'B23025_003E', 'B25010_001E', 'B25077_001E', 'B25064_001E', 'B28002_013E', 'B28002_002E', 'B25040_008E', 'Num_under_18', 'Num_18_to_34', 'Num_35_to_64', 'Num_65_more', 'Num_industry_agriculture_fishing_hunting']


In [20]:
new_names = ['County_name','State_name','County_code','Year','Total_population','Total_households','Female_population','Male_population', 'Median_age','Median_household_income_past_12m','Bachelor_degree','Master_degree', 'Professional_degree','Doctorate_degree','Num_employed','Num_unemployed','Labor_force','Avg_household_size','Median_value_owner_occupied_housing', 'Median_gross_rent','Households_no_internet','Households_internet_subscription','Households_solar_energy_fuel','Num_under_18','Num_18_to_34', 'Num_35_to_64','Num_65_more', 'Num_industry_agriculture_fishing_hunting']

In [21]:
census_df_variables.columns=new_names

checking that indeed all the rows of each age group match the total population

In [22]:
census_df_variables['Calculated_Total'] = census_df_variables[['Num_under_18', 'Num_18_to_34', 'Num_35_to_64', 'Num_65_more']].sum(axis=1)

# New column that indicates whether 'Total_population' equals 'Calculated_Total'
census_df_variables['Total_matches'] = np.isclose(census_df_variables['Total_population'], census_df_variables['Calculated_Total'])

# Check how many rows match
match_count = census_df_variables['Total_matches'].sum()
print(f"Number of rows where calculated total matches 'Total_population': {match_count}")

# Check how many rows don't match
non_match_count = len(census_df_variables) - match_count
print(f"Number of rows where calculated total does not match 'Total_population': {non_match_count}")

Number of rows where calculated total matches 'Total_population': 638
Number of rows where calculated total does not match 'Total_population': 0


In [23]:
census_df_variables.to_csv('census_df_county.csv',index=False)

In [24]:
census_df_variables.Year.unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

# Climate Change American Minds (CCAM)

In [25]:
opinions_climatechange = pd.read_spss('/project/CCAM SPSS Data 2008-2022.sav')
opinions_climatechange

,case_ID,wave,year,weight_wave,weight_aggregate,happening,cause_original,cause_other_text,cause_recoded,sci_consensus,...,employment,house_head,house_size,house_ages0to1,house_ages2to5,house_ages6to12,house_ages13to17,house_ages18plus,house_type,house_own
0,2.0,Nov 2008,2008,0.5400,0.269626,Yes,Caused mostly by human activities,,Caused mostly by human activities,Most scientists think global warming is happening,...,Not working - retired,Not head of household,3.0,0.0,0.0,0.0,0.0,3.0,One-family house detached from any other house,Owned by you or someone in your household
1,3.0,Nov 2008,2008,0.8500,0.424411,Don't know,Caused mostly by human activities,,Caused mostly by human activities,Don't know enough to say,...,Not working - disabled,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,Mobile home,Rented
2,5.0,Nov 2008,2008,0.4900,0.244660,Don't know,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,...,Not working - looking for work,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
3,6.0,Nov 2008,2008,0.2900,0.144799,Yes,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,...,Not working - retired,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
4,7.0,Nov 2008,2008,1.2900,0.644106,Yes,Caused mostly by human activities,,Caused mostly by human activities,There is a lot of disagreement among scientist...,...,Working - as a paid employee,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30131,53046.0,Apr 2022,2022,1.4848,1.575959,Don't know,Caused mostly by natural changes in the enviro...,-1,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
30132,53047.0,Apr 2022,2022,0.8351,0.886371,Yes,Caused mostly by natural changes in the enviro...,-1,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
30133,53048.0,Apr 2022,2022,1.0979,1.165305,Yes,Caused mostly by natural changes in the enviro...,-1,Caused mostly by natural changes in the enviro...,Don't know enough to say,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,Rented
30134,53049.0,Apr 2022,2022,0.6474,0.687147,No,Caused mostly by natural changes in the enviro...,-1,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,Owned by you or someone in your household


In [26]:
opinions_climatechange.columns

Index(['case_ID', 'wave', 'year', 'weight_wave', 'weight_aggregate',
       'happening', 'cause_original', 'cause_other_text', 'cause_recoded',
       'sci_consensus', 'worry', 'harm_personally', 'harm_US',
       'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
       'when_harm_US', 'reg_CO2_pollutant', 'reg_utilities', 'fund_research',
       'reg_coal_emissions', 'discuss_GW', 'hear_GW_media', 'gender', 'age',
       'age_category', 'generation', 'educ', 'educ_category', 'income',
       'income_category', 'race', 'ideology', 'party', 'party_w_leaners',
       'party_x_ideo', 'registered_voter', 'region9', 'region4', 'religion',
       'religion_other_nonchristian', 'evangelical', 'service_attendance',
       'marit_status', 'employment', 'house_head', 'house_size',
       'house_ages0to1', 'house_ages2to5', 'house_ages6to12',
       'house_ages13to17', 'house_ages18plus', 'house_type', 'house_own'],
      dtype='object')

In [27]:
opinions_climatechange.harm_US

0        A moderate amount
1                  Refused
2               Not at all
3            Only a little
4               Don't know
               ...        
30131         A great deal
30132        Only a little
30133         A great deal
30134        Only a little
30135    A moderate amount
Name: harm_US, Length: 30136, dtype: category
Categories (6, object): ['A great deal', 'A moderate amount', 'Don't know', 'Not at all', 'Only a little', 'Refused']

No steps necessary to collect this dataset. The dataset downloaded can be simply opened. 

# Yale climate change opinions map (YCCOM)

In [28]:
df_yale_cc = pd.read_excel("/project/ycom.coot.2008-2021.xlsx", sheet_name="ycom2014-2021.coot2008-2020")

In [29]:
df_yale_cc

,varname,GEOID,GeoName,GeoType,2008,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,congress,1.0,Alabama,State,58.981908,49.802614,49.765859,53.228234,51.458185,47.786980,53.212674,52.105523,54.264808,57.941226,54.301949,54.258945,54.046
1,congress,2.0,Alaska,State,60.132746,46.002393,49.131080,51.257517,50.987592,49.746340,54.457395,55.638666,58.447552,59.277305,57.337735,55.995996,57.283
2,congress,4.0,Arizona,State,61.998327,50.883474,52.868549,55.486015,52.692013,52.763085,56.921733,58.611543,60.467437,61.793629,59.464964,58.594975,60.166
3,congress,5.0,Arkansas,State,59.723048,48.937187,48.386756,52.154763,50.748722,48.179800,53.114320,52.970622,55.432803,57.857793,55.320774,54.950286,56.360
4,congress,6.0,California,State,65.641937,55.107412,57.686804,62.078066,61.100128,59.067086,64.717127,68.530911,67.226963,68.363890,66.629989,62.498390,65.822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403150,worriedOppose,5505.0,"Congressional District 5 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.586000,43.357
403151,worriedOppose,5506.0,"Congressional District 6 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.809000,42.171
403152,worriedOppose,5507.0,"Congressional District 7 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.243000,41.704
403153,worriedOppose,5508.0,"Congressional District 8 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.496000,44.799


In [30]:
df_yale_cc.GeoType.unique()

array(['State', 'County', 'cd113', 'National', 'CBSA', 'cd115', 'cd116'],
      dtype=object)

In [31]:
df_yale_cc

,varname,GEOID,GeoName,GeoType,2008,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,congress,1.0,Alabama,State,58.981908,49.802614,49.765859,53.228234,51.458185,47.786980,53.212674,52.105523,54.264808,57.941226,54.301949,54.258945,54.046
1,congress,2.0,Alaska,State,60.132746,46.002393,49.131080,51.257517,50.987592,49.746340,54.457395,55.638666,58.447552,59.277305,57.337735,55.995996,57.283
2,congress,4.0,Arizona,State,61.998327,50.883474,52.868549,55.486015,52.692013,52.763085,56.921733,58.611543,60.467437,61.793629,59.464964,58.594975,60.166
3,congress,5.0,Arkansas,State,59.723048,48.937187,48.386756,52.154763,50.748722,48.179800,53.114320,52.970622,55.432803,57.857793,55.320774,54.950286,56.360
4,congress,6.0,California,State,65.641937,55.107412,57.686804,62.078066,61.100128,59.067086,64.717127,68.530911,67.226963,68.363890,66.629989,62.498390,65.822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403150,worriedOppose,5505.0,"Congressional District 5 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.586000,43.357
403151,worriedOppose,5506.0,"Congressional District 6 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.809000,42.171
403152,worriedOppose,5507.0,"Congressional District 7 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.243000,41.704
403153,worriedOppose,5508.0,"Congressional District 8 (116th Congress), Wis...",cd116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.496000,44.799


In [32]:
df_yale_cc_ref = pd.read_excel("/project/ycom.coot.2008-2021.xlsx", sheet_name="ycom.coot.meta")

In [33]:
df_yale_cc_ref

,varname,vardesc
0,affectweather,Estimated percentage who somewhat/strongly agr...
1,affectweatherOppose,Estimated percentage who somewhat/strongly dis...
2,citizens,Estimated percentage who think citizens themse...
3,citizensOppose,Estimated percentage who think citizens themse...
4,CO2limits,Estimated percentage who somewhat/strongly sup...
...,...,...
63,timingOppose,Estimated percentage who think global warming ...
64,trustclimsciSST,Estimated percentage who somewhat or strongly ...
65,trustclimsciSSTOppose,Estimated percentage who somewhat or strongly ...
66,worried,Estimated percentage who are somewhat/very wor...


No steps necessary to collect this dataset. The dataset downloaded can be simply opened. 